# 计算传播范围

In [2]:
WORKSPACEFOLDER = 'E:\Projects\ComplexNetwork'
import os
os.chdir(WORKSPACEFOLDER)

import config

DB = config.get_DB()

In [3]:
import json

import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse

In [4]:
import src.main.util.spread as spread

## DiSF_a

In [5]:
g_name, beta, mean= 'DiSF_a', 0.1000, 4

In [6]:
base_path = os.path.join(DB, g_name)
seed_path = os.path.join(base_path, 'seed')
ss_path = os.path.join(base_path, 'ss')
gin_mat_path = os.path.join(base_path, 'gin_mat')

criterions = ['out_degree', 'k_shell', 'closeness', 'betweenness', 'ci']